In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### 1. Importar el dataset

In [8]:
bmw_original = pd.read_csv('dataset/bmw_pricing_v3.csv')

### 2. Información general del dataframe

In [9]:
bmw_original.head()

,marca,modelo,km,potencia,fecha_registro,tipo_gasolina,color,tipo_coche,volante_regulable,aire_acondicionado,camara_trasera,asientos_traseros_plegables,elevalunas_electrico,bluetooth,gps,alerta_lim_velocidad,precio,fecha_venta
0,NaN,118,140411.0,100.0,2012-02-01,diesel,black,NaN,True,True,False,NaN,True,NaN,True,NaN,11300.0,2018-01-01
1,BMW,M4,13929.0,317.0,NaN,petrol,grey,convertible,True,True,False,NaN,False,True,True,True,69700.0,2018-02-01
2,BMW,320,183297.0,120.0,2012-04-01,diesel,white,NaN,False,False,False,NaN,True,False,True,False,10200.0,2018-02-01
3,BMW,420,128035.0,135.0,NaN,diesel,red,convertible,True,True,False,NaN,True,True,True,NaN,25100.0,2018-02-01
4,BMW,425,97097.0,160.0,NaN,diesel,silver,NaN,True,True,False,False,False,True,True,True,33400.0,2018-04-01


In [10]:
bmw_original.shape

(4843, 18)

In [11]:
bmw_original.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4843 entries, 0 to 4842
Data columns (total 18 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   marca                        3873 non-null   object 
 1   modelo                       4840 non-null   object 
 2   km                           4841 non-null   float64
 3   potencia                     4842 non-null   float64
 4   fecha_registro               2420 non-null   object 
 5   tipo_gasolina                4838 non-null   object 
 6   color                        4398 non-null   object 
 7   tipo_coche                   3383 non-null   object 
 8   volante_regulable            4839 non-null   object 
 9   aire_acondicionado           4357 non-null   object 
 10  camara_trasera               4841 non-null   object 
 11  asientos_traseros_plegables  1452 non-null   object 
 12  elevalunas_electrico         4841 non-null   object 
 13  bluetooth         

#### Primeras observaciones:
 - Las columnas aparentemente booleanas que tienen nulos (volante_regulable, aire_acondicionado...) se cargan como 	tipo object

 - La columna gps es booleana porque no tiene nulos
 
 - Las columnas de fecha (fecha_registro, fecha_venta) se cargan como object

### 3. Duplicados

In [15]:
bmw_original[bmw_original.duplicated()].shape[0]

0

Aparentemente, no hay registros duplicados. Compruebo que no existe ningún modelo con la misma fecha de venta y número de km repetidos 

In [23]:
bmw_original[bmw_original[['modelo', 'fecha_venta', 'km']].duplicated(keep=False)].sort_values(by=['fecha_venta', 'modelo']).shape[0]

0

Por lo tanto, no hay duplicados

### 4. Nulos

In [32]:
bmw_original.isnull().sum()

marca                           970
modelo                            3
km                                2
potencia                          1
fecha_registro                 2423
tipo_gasolina                     5
color                           445
tipo_coche                     1460
volante_regulable                 4
aire_acondicionado              486
camara_trasera                    2
asientos_traseros_plegables    3391
elevalunas_electrico              2
bluetooth                       728
gps                               0
alerta_lim_velocidad            728
precio                            6
fecha_venta                       1
dtype: int64

### 4.1 Columnas con muchos nulos

In [96]:
# Creo copia del df para tratar los nulos

bmw4_1 = bmw_original.copy()

In [97]:
# Muestro las columnas que tienen más del 20% de valores nulos

columnas_MasNulos = []

for col in bmw4_1.columns:
    if ((bmw4_1[col].isnull().sum() / bmw4_1.shape[0]) * 100 >= 20):
        columnas_MasNulos.append(col)

print(columnas_MasNulos)

['marca', 'fecha_registro', 'tipo_coche', 'asientos_traseros_plegables']


### 4.1.1 Columna marca

In [77]:
bmw4_1_1 = bmw4_1.copy()

In [81]:
round(bmw4_1_1['marca'].value_counts(dropna=False, normalize=True) * 100, 2)

marca
BMW    79.97
NaN    20.03
Name: proportion, dtype: float64

In [82]:
bmw4_1_1[bmw4_1_1['marca'].isnull()]['modelo'].value_counts()

modelo
320                  142
520                  123
318                  116
X3                   105
116                   78
X1                    55
X5                    49
316                   45
525                   34
118                   30
530                   29
318 Gran Turismo      19
518                   14
320 Gran Turismo      13
535                    9
420                    8
X4                     8
X6                     7
420 Gran Coupé         7
640 Gran Coupé         6
325                    6
730                    6
120                    6
X5 M                   5
530 Gran Turismo       4
M550                   4
X6 M                   3
114                    3
330                    3
M235                   3
740                    2
218                    2
435 Gran Coupé         2
125                    2
520 Gran Turismo       2
418 Gran Coupé         2
Z4                     2
435                    2
218 Active Tourer      2
X5 M50            

Esta columna no es significativa. Todos los modelos son de la marca BMW. Por lo tanto se elimina

In [83]:
del(bmw4_1_1['marca'])

In [84]:
bmw4_1_1.isnull().sum()

modelo                            3
km                                2
potencia                          1
fecha_registro                 2423
tipo_gasolina                     5
color                           445
tipo_coche                     1460
volante_regulable                 4
aire_acondicionado              486
camara_trasera                    2
asientos_traseros_plegables    3391
elevalunas_electrico              2
bluetooth                       728
gps                               0
alerta_lim_velocidad            728
precio                            6
fecha_venta                       1
dtype: int64

### 4.1.2 Columna fecha_registro

In [85]:
bmw4_1_2 = bmw4_1_1.copy()

In [86]:
round(bmw4_1_2['fecha_registro'].value_counts(dropna=False, normalize=True) * 100, 2)

fecha_registro
NaN           50.03
2013-07-01     1.92
2014-03-01     1.71
2014-05-01     1.67
2013-01-01     1.61
              ...  
2007-01-01     0.02
1990-03-01     0.02
2006-11-01     0.02
1994-01-01     0.02
2005-09-01     0.02
Name: proportion, Length: 181, dtype: float64

In [87]:
bmw4_1_2[bmw4_1_2['fecha_registro'].isnull()]

,modelo,km,potencia,fecha_registro,tipo_gasolina,color,tipo_coche,volante_regulable,aire_acondicionado,camara_trasera,asientos_traseros_plegables,elevalunas_electrico,bluetooth,gps,alerta_lim_velocidad,precio,fecha_venta
1,M4,13929.0,317.0,NaN,petrol,grey,convertible,True,True,False,NaN,False,True,True,True,69700.0,2018-02-01
3,420,128035.0,135.0,NaN,diesel,red,convertible,True,True,False,NaN,True,True,True,NaN,25100.0,2018-02-01
4,425,97097.0,160.0,NaN,diesel,silver,NaN,True,True,False,False,False,True,True,True,33400.0,2018-04-01
6,325,205219.0,145.0,NaN,diesel,grey,convertible,True,True,False,NaN,True,True,True,True,12400.0,2018-02-01
8,Z4,123886.0,125.0,NaN,petrol,black,convertible,True,False,False,False,False,NaN,True,False,6200.0,2018-03-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4834,218 Gran Tourer,47782.0,110.0,NaN,diesel,blue,NaN,False,NaN,False,NaN,False,False,True,True,14900.0,2018-07-01
4835,218 Active Tourer,165707.0,110.0,NaN,diesel,black,van,False,True,False,NaN,False,NaN,True,True,13600.0,2018-07-01
4837,218 Gran Tourer,66770.0,110.0,NaN,diesel,blue,van,False,True,False,NaN,False,False,True,False,13400.0,2018-07-01
4838,218 Gran Tourer,39743.0,110.0,NaN,diesel,black,NaN,False,True,False,NaN,False,False,True,False,14600.0,2018-08-01


Considero que esta columna no es muy significativa y contiene demasiados valores nulos. Por tanto, la elimino

In [88]:
del(bmw4_1_2['fecha_registro'])

In [89]:
bmw4_1_2.isnull().sum()

modelo                            3
km                                2
potencia                          1
tipo_gasolina                     5
color                           445
tipo_coche                     1460
volante_regulable                 4
aire_acondicionado              486
camara_trasera                    2
asientos_traseros_plegables    3391
elevalunas_electrico              2
bluetooth                       728
gps                               0
alerta_lim_velocidad            728
precio                            6
fecha_venta                       1
dtype: int64

### 4.1.3 Columna tipo_coche

In [90]:
bmw4_1_3 = bmw4_1_2.copy()

In [91]:
round(bmw4_1_3['tipo_coche'].value_counts(dropna=False, normalize=True) * 100, 2)

tipo_coche
NaN            30.15
estate         22.86
sedan          16.95
suv            15.59
hatchback      10.08
subcompact      1.59
coupe           1.57
convertible     0.62
van             0.60
Name: proportion, dtype: float64

Esta columna tiene bastantes nulos, pero considero que es representativa. Existen distintos valores con porcentajes altos

No la elimino. De momento se queda así, pero más adelante le daré un valor a los nulos.

### 4.1.4 Columna asientos_traseros_plegables

In [98]:
bmw4_1_4 = bmw4_1_3.copy()

In [99]:
round(bmw4_1_4['asientos_traseros_plegables'].value_counts(dropna=False, normalize=True) * 100, 2)

asientos_traseros_plegables
NaN      70.02
False    23.75
True      6.24
Name: proportion, dtype: float64

Esta columna es de tipo booleano y tiene demasiados valores nulos. La elimino

In [100]:
del(bmw4_1_4['asientos_traseros_plegables'])

In [101]:
bmw4_1_4.isnull().sum()

modelo                     3
km                         2
potencia                   1
tipo_gasolina              5
color                    445
tipo_coche              1460
volante_regulable          4
aire_acondicionado       486
camara_trasera             2
elevalunas_electrico       2
bluetooth                728
gps                        0
alerta_lim_velocidad     728
precio                     6
fecha_venta                1
dtype: int64

### 4.2 Registros con varios nulos

In [102]:
bmw4_2 = bmw4_1_4.copy()

In [105]:
# Creo una lista para almacenar los índices de los registros que tienen 2 o más nulos
registros_2Nulos = []

for i in bmw4_2.index:
    if (bmw4_2.loc[i, :].isnull().sum() >= 2):
        registros_2Nulos.append(i)

print(registros_2Nulos, end='\n\n')
print(f"Hay {len(registros_2Nulos)} registros con más de 1 valor nulo -> {round(len(registros_2Nulos) / bmw4_2.shape[0]  * 100,2)} %")

[0, 11, 12, 18, 23, 26, 33, 34, 36, 41, 45, 49, 56, 63, 64, 66, 81, 82, 86, 92, 100, 106, 117, 119, 128, 130, 134, 137, 138, 141, 148, 149, 155, 168, 169, 173, 177, 182, 183, 199, 202, 203, 206, 217, 225, 230, 237, 244, 257, 261, 263, 273, 276, 281, 291, 297, 302, 308, 310, 314, 317, 318, 321, 323, 327, 328, 333, 335, 336, 348, 349, 358, 360, 370, 373, 374, 376, 385, 398, 409, 412, 429, 434, 442, 450, 452, 456, 459, 464, 474, 476, 477, 480, 485, 489, 503, 505, 507, 509, 511, 512, 513, 528, 537, 538, 541, 544, 549, 558, 561, 563, 569, 570, 584, 587, 594, 599, 600, 601, 609, 611, 614, 615, 620, 630, 641, 645, 649, 657, 662, 668, 672, 684, 689, 691, 692, 693, 697, 698, 711, 723, 733, 737, 747, 751, 753, 758, 766, 778, 780, 791, 809, 819, 820, 824, 827, 849, 851, 852, 855, 863, 868, 873, 880, 883, 886, 890, 893, 895, 896, 899, 902, 914, 928, 930, 932, 946, 951, 953, 955, 957, 959, 964, 973, 974, 979, 981, 985, 994, 998, 1013, 1016, 1019, 1033, 1036, 1040, 1043, 1045, 1049, 1051, 1064, 1067

In [106]:
bmw4_2.loc[registros_2Nulos, :]

,modelo,km,potencia,tipo_gasolina,color,tipo_coche,volante_regulable,aire_acondicionado,camara_trasera,elevalunas_electrico,bluetooth,gps,alerta_lim_velocidad,precio,fecha_venta
0,118,140411.0,100.0,diesel,black,NaN,True,True,False,True,NaN,True,NaN,11300.0,2018-01-01
11,325,228000.0,145.0,diesel,black,NaN,True,True,False,True,False,True,NaN,13300.0,2018-03-01
12,420,132025.0,135.0,diesel,blue,NaN,True,NaN,False,True,True,True,NaN,21700.0,2018-03-01
18,325,205474.0,145.0,diesel,NaN,NaN,True,True,False,True,NaN,True,NaN,11000.0,2018-04-01
23,220,46963.0,140.0,diesel,orange,NaN,False,True,False,False,NaN,True,True,23300.0,2018-05-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4818,X3,110039.0,130.0,diesel,NaN,NaN,False,NaN,False,False,False,True,False,1300.0,2018-05-01
4820,Active Tourer,48380.0,100.0,diesel,black,van,True,NaN,False,False,NaN,True,False,19000.0,2018-05-01
4830,216 Gran Tourer,48012.0,85.0,diesel,blue,NaN,True,True,False,False,True,True,NaN,15600.0,2018-06-01
4834,218 Gran Tourer,47782.0,110.0,diesel,blue,NaN,False,NaN,False,False,False,True,True,14900.0,2018-07-01


Hay demasiados registros, no podemos eliminarlos todos. Voy a filtrar por las columnas más representativas (a mi criterio)

In [107]:
col_significativas = ['modelo', 'km', 'potencia', 'precio', 'fecha_venta']

In [108]:
registros_2Nulos_Significativos = []

for i in bmw4_2.index:
    if (bmw4_2.loc[i, col_significativas].isnull().sum() >= 2):
        registros_2Nulos.append(i)

print(registros_2Nulos_Significativos)
print(f"Hay {len(registros_2Nulos_Significativos)} registros con más de 1 valor significativo nulo")

[]
Hay 0 registros con más de 1 valor significativo nulo


Como no existen registros con más de una columna significativa nula, voy a ver que numero de registros tienen 3, 4 y 5 valores nulos. 

Si el porcentaje respecto al total de df es pequeño, elimino estos registros

In [109]:
# Creo una lista para almacenar los índices de los registros que tienen 2 o más nulos
registros_MasNulos = {
    3 : [],
    4 : [],
    5 : []
}

for nulos in range(3, 6):
    for i in bmw4_2.index:
        if (bmw4_2.loc[i, :].isnull().sum() >= nulos):
            registros_MasNulos[nulos].append(i)

print(registros_MasNulos, end='\n\n')
for key, value in registros_MasNulos.items():
    print(f"Hay {len(value)} registros con {key} valores nulos \t->\t{round((len(value) / bmw4_2.shape[0]) * 100, 1)}\t%")

{3: [0, 12, 18, 26, 49, 134, 173, 182, 217, 230, 273, 291, 409, 412, 442, 450, 485, 505, 513, 614, 691, 692, 693, 697, 711, 824, 849, 852, 880, 895, 914, 955, 973, 974, 981, 1013, 1043, 1051, 1067, 1131, 1141, 1186, 1195, 1229, 1258, 1452, 1491, 1515, 1544, 1546, 1587, 1605, 1660, 1689, 1690, 1730, 1742, 1759, 1895, 1911, 1914, 1998, 2001, 2027, 2048, 2059, 2165, 2206, 2254, 2395, 2437, 2473, 2514, 2537, 2545, 2609, 2627, 2629, 2641, 2650, 2713, 2742, 2791, 2796, 2811, 2856, 2899, 2921, 2968, 3018, 3027, 3035, 3156, 3165, 3176, 3228, 3240, 3389, 3394, 3399, 3401, 3412, 3498, 3573, 3593, 3722, 3732, 3735, 3749, 3787, 3878, 3973, 3976, 4000, 4040, 4055, 4056, 4188, 4260, 4316, 4379, 4463, 4466, 4474, 4476, 4510, 4534, 4549, 4618, 4715, 4757, 4784, 4793, 4805, 4818], 4: [18, 173, 697, 711, 1043, 1742, 2027, 2165, 2791, 2968, 4379], 5: []}

Hay 135 registros con 3 valores nulos 	->	2.8	%
Hay 11 registros con 4 valores nulos 	->	0.2	%
Hay 0 registros con 5 valores nulos 	->	0.0	%


Voy a evaluar la eliminación de los registros con 3 nulos

In [110]:
bmw4_2.loc[registros_MasNulos[3], :]

,modelo,km,potencia,tipo_gasolina,color,tipo_coche,volante_regulable,aire_acondicionado,camara_trasera,elevalunas_electrico,bluetooth,gps,alerta_lim_velocidad,precio,fecha_venta
0,118,140411.0,100.0,diesel,black,NaN,True,True,False,True,NaN,True,NaN,11300.0,2018-01-01
12,420,132025.0,135.0,diesel,blue,NaN,True,NaN,False,True,True,True,NaN,21700.0,2018-03-01
18,325,205474.0,145.0,diesel,NaN,NaN,True,True,False,True,NaN,True,NaN,11000.0,2018-04-01
26,430,113678.0,190.0,diesel,black,NaN,True,True,False,True,NaN,True,NaN,30100.0,2018-05-01
49,118,147180.0,100.0,diesel,NaN,coupe,True,NaN,True,False,False,True,NaN,8400.0,2018-07-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4757,X4,61989.0,140.0,diesel,NaN,NaN,False,False,False,False,NaN,False,True,28600.0,2018-09-01
4784,X1,120600.0,85.0,diesel,black,NaN,False,NaN,True,False,False,False,NaN,11100.0,2018-09-01
4793,X5 M,125197.0,230.0,diesel,blue,NaN,True,NaN,True,True,True,True,NaN,42700.0,2018-09-01
4805,218 Active Tourer,35382.0,110.0,diesel,NaN,van,False,NaN,False,False,NaN,True,True,13100.0,2018-02-01


Se observa que los valores nulos no se corresponden con las columnas más significativas

Eliminar estos registros me parece perder información. Por lo tanto, no los elimino.

Evaluo los registros con 4 o más valores nulos

In [111]:
bmw4_2.loc[registros_MasNulos[4], :]

,modelo,km,potencia,tipo_gasolina,color,tipo_coche,volante_regulable,aire_acondicionado,camara_trasera,elevalunas_electrico,bluetooth,gps,alerta_lim_velocidad,precio,fecha_venta
18,325,205474.0,145.0,diesel,NaN,NaN,True,True,False,True,NaN,True,NaN,11000.0,2018-04-01
173,NaN,146338.0,105.0,diesel,black,NaN,False,True,False,False,NaN,True,NaN,13300.0,2018-08-01
697,318,146248.0,100.0,diesel,NaN,NaN,True,True,True,False,NaN,True,NaN,11300.0,2018-03-01
711,530,223328.0,190.0,diesel,black,NaN,True,NaN,True,False,NaN,True,NaN,15800.0,2018-03-01
1043,520,172414.0,120.0,diesel,NaN,NaN,True,True,False,False,NaN,True,NaN,15400.0,2018-05-01
1742,318,78430.0,100.0,diesel,NaN,NaN,True,NaN,False,False,NaN,True,False,15500.0,2018-09-01
2027,116,186903.0,85.0,diesel,NaN,NaN,False,NaN,False,False,NaN,True,False,10200.0,2018-08-01
2165,435 Gran Coupé,64985.0,230.0,diesel,NaN,NaN,True,True,True,False,NaN,True,NaN,34500.0,2018-05-01
2791,118,105764.0,105.0,diesel,NaN,NaN,False,False,False,False,NaN,True,NaN,10800.0,2018-03-01
2968,535,153039.0,230.0,diesel,NaN,NaN,True,NaN,False,True,False,True,NaN,23000.0,2018-04-01


Como son pocos registros, los elimino.

In [113]:
bmw4_2.drop(registros_MasNulos[4], inplace=True)

In [114]:
bmw4_2.isnull().sum()

modelo                     2
km                         2
potencia                   1
tipo_gasolina              5
color                    436
tipo_coche              1449
volante_regulable          4
aire_acondicionado       482
camara_trasera             2
elevalunas_electrico       2
bluetooth                718
gps                        0
alerta_lim_velocidad     719
precio                     6
fecha_venta                1
dtype: int64

### 4.3 Columnas con nulos

In [116]:
bmw4_3 = bmw4_2.copy()

In [118]:
columnas_Nulos = []

for col in bmw4_3.columns:
    if (bmw4_3[col].isnull().any()):
        columnas_Nulos.append(col)

columnas_Nulos

['modelo',
 'km',
 'potencia',
 'tipo_gasolina',
 'color',
 'tipo_coche',
 'volante_regulable',
 'aire_acondicionado',
 'camara_trasera',
 'elevalunas_electrico',
 'bluetooth',
 'alerta_lim_velocidad',
 'precio',
 'fecha_venta']

Hay que vaciar esta lista

In [119]:
bmw4_3.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4832 entries, 0 to 4842
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   modelo                4830 non-null   object 
 1   km                    4830 non-null   float64
 2   potencia              4831 non-null   float64
 3   tipo_gasolina         4827 non-null   object 
 4   color                 4396 non-null   object 
 5   tipo_coche            3383 non-null   object 
 6   volante_regulable     4828 non-null   object 
 7   aire_acondicionado    4350 non-null   object 
 8   camara_trasera        4830 non-null   object 
 9   elevalunas_electrico  4830 non-null   object 
 10  bluetooth             4114 non-null   object 
 11  gps                   4832 non-null   bool   
 12  alerta_lim_velocidad  4113 non-null   object 
 13  precio                4826 non-null   float64
 14  fecha_venta           4831 non-null   object 
dtypes: bool(1), float64(3), ob

### 4.3.1 modelo

### 4.3.2 km

### 4.3.3 potencia

### 4.3.4 tipo_gasolina

### 4.3.5 color

### 4.3.6 tipo_coche

### 4.3.7 volante_regulable

### 4.3.8 aire_acondicionado

### 4.3.9 camara_trasera

### 4.3.10 elevalunas_electrico

### 4.3.11 bluetooth

### 4.3.12 gps

### 4.3.13 alerta_lim_velocidad

### 4.3.14 precio

### 4.3.14 fecha_venta

### 5. Análisis univariable